In [4]:
import numpy as np
import pandas as pd

In [2]:
pip install scikit_surprise

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 771 kB 104 kB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-macosx_10_9_universal2.whl size=1607910 sha256=52cdce9b2f98176323f7eaabd769d23f9f968104b941d3051a41fbcaf490abcf
  Stored in directory: /Users/changlu/Library/Caches/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

In [6]:
ratings = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
data = pd.merge(ratings, movies, on = 'movieId')

In [10]:
reader = Reader(rating_scale=(0.5,5))

In [11]:
data = Dataset.load_from_df(data[['userId', 'movieId','rating']], reader)

In [12]:
data

In [13]:
params = {'name' : 'cosine', 'user_based': False}

In [14]:
model = KNNBasic(sim_options=params)

In [15]:
cross_validate(model, data, measures = ['RMSE', 'MAE'], cv =5 , verbose= True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9675  0.9727  0.9734  0.9719  0.9804  0.9732  0.0042  
MAE (testset)     0.7528  0.7553  0.7577  0.7538  0.7631  0.7565  0.0037  
Fit time          13.33   9.88    9.63    9.64    11.15   10.73   1.42    
Test time         12.00   12.54   14.69   14.58   14.91   13.74   1.22    


{'test_rmse': array([0.96748869, 0.97271221, 0.97339114, 0.97187188, 0.98039027]),
 'test_mae': array([0.75281246, 0.75527204, 0.75772129, 0.75379303, 0.76314622]),
 'fit_time': (13.331189155578613,
  9.876800060272217,
  9.631531000137329,
  9.636304140090942,
  11.154652833938599),
 'test_time': (12.004442930221558,
  12.541460037231445,
  14.685264825820923,
  14.583992004394531,
  14.907461881637573)}

In [16]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [17]:
trainset, testset = train_test_split(data, test_size = 0.2)

In [22]:
model2 = KNNBasic(min_k = 5, k=20, sim_options={'user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9350


0.9350342012326683

In [24]:
model2 = KNNBasic(min_k = 5, k=10, sim_options={'name':'msd','user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9369


0.9368807648949601

In [25]:
model2.predict(668,16)

Prediction(uid=668, iid=16, r_ui=None, est=3.6408588493815395, details={'actual_k': 10, 'was_impossible': False})

In [28]:
movies[movies['movieId']==16]

,movieId,title,genres
15,16,Casino (1995),Crime|Drama


In [29]:
final_dataset = pd.merge(movies,ratings, on='movieId')

In [30]:
final_dataset['predicted_rating'] = final_dataset.apply(lambda x:model2.predict(x['userId'],x['movieId']).est, axis=1)

In [31]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,3.748277
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.000000
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,4.912088
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.850044
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.000000


In [33]:
#svd
from surprise import SVD